In [15]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
from scipy.spatial import KDTree
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
# %matplotlib inline

In [112]:
#this is borrowed from https://github.com/davideverona/deep-crowd-counting_crowdnet
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.nonzero(gt)
    pts = np.vstack([pts[1], pts[0]]).T

    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(pts, leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

    print('generate density...')
    for i, pt in enumerate(pts):
        print(i, pt)
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = s1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print('done.')
    return density

In [113]:
#set the root to the Shanghai dataset you download
root = '/ktmp/ShanghaiTech_Crowd_Counting_Dataset'

In [114]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_train,part_A_test]

In [115]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [116]:
for img_path in img_paths:
    print(img_path)
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    img = plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
#     print(np.sum(ㄋk))
    k = gaussian_filter_density(k)
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground_truth'), 'w') as hf:
        hf['density'] = k
    break

/ktmp/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/train_data/images/IMG_247.jpg
(681, 1024)
(588, 2)
generate density...
0 [367   6]
1 [21  7]
2 [294   7]
3 [634   7]
4 [654   7]
5 [ 9 10]
6 [212  10]
7 [460  10]
8 [352  11]
9 [533  11]
10 [603  11]
11 [114  12]
12 [435  12]
13 [33 13]
14 [379  13]
15 [391  13]
16 [504  13]
17 [52 14]
18 [151  14]
19 [325  14]
20 [89 15]
21 [178  15]
22 [270  15]
23 [398  15]
24 [523  15]
25 [299  16]
26 [247  17]
27 [465  17]
28 [135  18]
29 [203  18]
30 [219  18]
31 [420  18]
32 [498  18]
33 [101  20]
34 [68 21]
35 [443  21]
36 [283  22]
37 [336  22]
38 [27 23]
39 [181  23]
40 [130  24]
41 [363  24]
42 [387  24]
43 [307  26]
44 [81 27]
45 [259  27]
46 [991  28]
47 [206  31]
48 [222  31]
49 [41 35]
50 [62 35]
51 [71 35]
52 [108  35]
53 [36 37]
54 [992  43]
55 [950  46]
56 [931  47]
57 [978  49]
58 [961  53]
59 [67 56]
60 [794  57]
61 [803  58]
62 [998  58]
63 [761  59]
64 [782  60]
65 [628  61]
66 [735  65]
67 [868  65]
68 [145  67]
69 [827  67]


587 [1006  666]
done.


In [121]:
# plt.imshow(k)
# np.max(k)
#now see a sample from ShanghaiA
# plt.imshow(Image.open(img_paths[0]))
img_path

'/ktmp/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/train_data/images/IMG_247.jpg'

In [ ]:
gt_file = h5py.File(img_paths[0].replace('.jpg','.h5').replace('images','ground_truth'),'r')
groundtruth = np.asarray(gt_file['density'])
plt.imshow(groundtruth,cmap=CM.jet)

In [ ]:
np.sum(groundtruth)# don't mind this slight variation

In [ ]:
#now generate the ShanghaiB's ground truth
path_sets = [part_B_train,part_B_test]

In [ ]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [ ]:
for img_path in img_paths:
    print img_path
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    img= plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
    k = gaussian_filter(k,15)
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground_truth'), 'w') as hf:
            hf['density'] = k